<a href="https://colab.research.google.com/github/mahanthnaidu/WIDS-3.0/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [20]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
layers = []
layers.append(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1))
layers.append(nn.ReLU())
layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
layers.append(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1))
layers.append(nn.ReLU())
layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
layers.append(nn.Flatten())
layers.append(nn.Linear(64 * 7 * 7, 128))
layers.append(nn.ReLU())
layers.append(nn.Dropout(0.3))
layers.append(nn.Linear(128, 10))
model = nn.Sequential(*layers)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
num_epochs = 5
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
model.to(device)

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=3136, out_features=128, bias=True)
  (8): ReLU()
  (9): Dropout(p=0.3, inplace=False)
  (10): Linear(in_features=128, out_features=10, bias=True)
)

In [23]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Step [{i+1}/{len(train_loader)}], "
                  f"Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

print("Training Done!")

Epoch [1/5], Step [100/1875], Loss: 0.6434
Epoch [1/5], Step [200/1875], Loss: 0.2587
Epoch [1/5], Step [300/1875], Loss: 0.2112
Epoch [1/5], Step [400/1875], Loss: 0.1703
Epoch [1/5], Step [500/1875], Loss: 0.1501
Epoch [1/5], Step [600/1875], Loss: 0.1391
Epoch [1/5], Step [700/1875], Loss: 0.1417
Epoch [1/5], Step [800/1875], Loss: 0.1318
Epoch [1/5], Step [900/1875], Loss: 0.1081
Epoch [1/5], Step [1000/1875], Loss: 0.1282
Epoch [1/5], Step [1100/1875], Loss: 0.1188
Epoch [1/5], Step [1200/1875], Loss: 0.1212
Epoch [1/5], Step [1300/1875], Loss: 0.1287
Epoch [1/5], Step [1400/1875], Loss: 0.1082
Epoch [1/5], Step [1500/1875], Loss: 0.1023
Epoch [1/5], Step [1600/1875], Loss: 0.1020
Epoch [1/5], Step [1700/1875], Loss: 0.1072
Epoch [1/5], Step [1800/1875], Loss: 0.0879
Epoch [2/5], Step [100/1875], Loss: 0.1027
Epoch [2/5], Step [200/1875], Loss: 0.1201
Epoch [2/5], Step [300/1875], Loss: 0.0838
Epoch [2/5], Step [400/1875], Loss: 0.0975
Epoch [2/5], Step [500/1875], Loss: 0.0729
Ep

In [24]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test dataset: {accuracy:.2f}%")

Accuracy on the test dataset: 98.32%
